In [28]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
from keras.datasets import mnist
from keras.utils import np_utils

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [30]:
X_train = X_train.reshape(-1, 28* 28).astype(np.float32)
X_test = X_test.reshape(-1, 28*28).astype(np.float32)
Y_train = np_utils.to_categorical(Y_train,10)
Y_test = np_utils.to_categorical(Y_test,10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


In [31]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

## Batch Normalization 성능 비교

In [35]:
model = keras.models.Sequential([
    keras.layers.Dense(32,input_dim=784),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(32),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dense(10,activation='softmax')
])

In [36]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                25120     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32)                128       
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32)                128       
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)               

In [41]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=["accuracy"])

In [44]:
history = model.fit(X_train, Y_train, epochs = 10, batch_size=32,
                    shuffle=True, verbose=0)

In [58]:
model2 = keras.models.Sequential([
    keras.layers.Dense(32,input_dim=784),
    keras.layers.Activation("relu"),
    keras.layers.Dense(32),
    keras.layers.Activation('relu'),
    keras.layers.Dense(10,activation='softmax')
])

model2.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

history2 = model2.fit(X_train, Y_train, epochs=10, batch_size=32,
                     shuffle=True, verbose=0)

In [59]:
bn_y = pd.DataFrame(history.history)
bn_n = pd.DataFrame(history2.history)

In [66]:
loss = pd.concat([bn_y[['loss']],bn_n[['loss']]],axis=1)
loss

,loss,loss
0,0.277670,1.915966
1,0.171482,1.440990
2,0.144047,1.432988
3,0.130332,1.435214
4,0.117637,1.594033
5,0.108688,1.639424
6,0.103545,1.656975
7,0.099168,1.612371
8,0.094668,1.653261
9,0.091196,1.600714


In [67]:
acc = pd.concat([bn_y[['accuracy']],bn_n[['accuracy']]],axis=1)
acc

,accuracy,accuracy
0,0.914750,0.362833
1,0.947033,0.408183
2,0.954300,0.382083
3,0.960483,0.377983
4,0.963617,0.319917
5,0.965817,0.295850
6,0.967417,0.293683
7,0.968017,0.295683
8,0.970333,0.293483
9,0.971600,0.291717


In [68]:
model.evaluate(X_test,Y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0963 - accuracy: 0.9714


[0.09633772075176239, 0.9714000225067139]

In [69]:
model2.evaluate(X_test,Y_test)

313/313 [==============================] - 0s 1ms/step - loss: 1.6023 - accuracy: 0.2949


[1.6023356914520264, 0.29490000009536743]

## drop out의 성능 비교

In [71]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(512, activation='relu',
                            kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(10,activation='softmax'))

In [72]:
model.compile(loss='categorical_crossentropy',
             optimizer=keras.optimizers.Adam(lr=0.01),
             metrics=['accuracy'])

In [74]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=32,
                   shuffle=True, verbose=0)

In [75]:
model2 = keras.models.Sequential()
model2.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model2.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model2.add(keras.layers.Dense(512,input_dim=784, activation='relu',
                            kernel_initializer='he_normal'))
model2.add(keras.layers.Dense(512, activation='relu',
                            kernel_initializer='he_normal'))
model2.add(keras.layers.Dense(10,activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

In [76]:
history2 = model2.fit(X_train, Y_train, epochs=10, batch_size=32,
                     shuffle=True, verbose=0)

In [78]:
hist1 = pd.DataFrame(history.history)
hist2 = pd.DataFrame(history2.history)

loss = pd.concat([hist1['loss'],hist2['loss']],axis=1)
loss

,loss,loss
0,6.049829,7.384924
1,2.367099,1.340108
2,3.311633,2.302533
3,2.313841,2.302559
4,2.612107,2.302700
5,2.306081,2.302584
6,2.389167,2.302637
7,2.497485,2.302448
8,2.309135,2.302672
9,2.302508,2.302657


In [79]:
acc = pd.concat([hist1['accuracy'],hist2['accuracy']],axis=1)
acc

,accuracy,accuracy
0,0.122383,0.577383
1,0.109100,0.552967
2,0.108933,0.109100
3,0.108950,0.110467
4,0.109650,0.109333
5,0.109067,0.109667
6,0.109917,0.109033
7,0.108483,0.109967
8,0.109617,0.107833
9,0.107450,0.109067
